我们建议将其tf.keras用作构建神经网络的高级API。也就是说，大多数TensorFlow API都可用于急切执行的环境。

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [2]:
import tensorflow as tf

层：常用的有用操作集<br>
在大多数时候，为机器学习模型编写代码时，您希望比单个操作和单个变量的操作具有更高的抽象水平。

许多机器学习模型可表示为相对简单的层的组成和堆叠，并且TensorFlow提供了许多常见层的集合以及简便的方法，使您可以从头开始或以自己的方式编写自己的特定于应用程序的层现有图层。

TensorFlow包括完整Keras在tf.keras封装API，并建立自己的模型时，Keras层是非常有用的。

In [3]:
# In the tf.keras.layers package, layers are objects. To construct a layer,
# simply construct the object. Most layers take as a first argument the number
# of output dimensions / channels.
layer = tf.keras.layers.Dense(100)
# The number of input dimensions is often unnecessary, as it can be inferred
# the first time the layer is used, but it can be provided if you want to
# specify it manually, which is useful in some complex models.
layer = tf.keras.layers.Dense(10, input_shape=(None, 5))

可以在文档中找到预先存在的层的完整列表。它包括Dense（完全连接的层），Conv2D，LSTM，BatchNormalization，Dropout等。

In [4]:
# To use a layer, simply call it.
layer(tf.zeros([10, 5]))

<tf.Tensor: id=29, shape=(10, 10), dtype=float32, numpy=
array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)>

In [5]:
layer.variables

[<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-0.45266646,  0.42345542,  0.03176934, -0.25730163, -0.16324735,
         -0.36828366,  0.20345676,  0.4696378 , -0.35339668,  0.3445425 ],
        [-0.57233286, -0.5415045 ,  0.5231716 , -0.21567446, -0.06638354,
          0.14920151, -0.21916494, -0.2507266 , -0.08368182, -0.13707411],
        [ 0.31754446,  0.56409544, -0.3010157 ,  0.34711117,  0.4657684 ,
          0.47865266,  0.19924647,  0.17497194,  0.58175534,  0.4511518 ],
        [ 0.4746551 , -0.01703632,  0.09176272, -0.43094528, -0.11845225,
          0.20634758,  0.43336815,  0.07344937,  0.20250648,  0.14471114],
        [-0.16020578,  0.38120311,  0.35114586, -0.3125559 , -0.630895  ,
         -0.06671935,  0.1690957 ,  0.39048976,  0.31340122, -0.16485384]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>]

In [6]:
# The variables are also accessible through nice accessors
layer.kernel, layer.bias

(<tf.Variable 'dense_1/kernel:0' shape=(5, 10) dtype=float32, numpy=
 array([[-0.45266646,  0.42345542,  0.03176934, -0.25730163, -0.16324735,
         -0.36828366,  0.20345676,  0.4696378 , -0.35339668,  0.3445425 ],
        [-0.57233286, -0.5415045 ,  0.5231716 , -0.21567446, -0.06638354,
          0.14920151, -0.21916494, -0.2507266 , -0.08368182, -0.13707411],
        [ 0.31754446,  0.56409544, -0.3010157 ,  0.34711117,  0.4657684 ,
          0.47865266,  0.19924647,  0.17497194,  0.58175534,  0.4511518 ],
        [ 0.4746551 , -0.01703632,  0.09176272, -0.43094528, -0.11845225,
          0.20634758,  0.43336815,  0.07344937,  0.20250648,  0.14471114],
        [-0.16020578,  0.38120311,  0.35114586, -0.3125559 , -0.630895  ,
         -0.06671935,  0.1690957 ,  0.39048976,  0.31340122, -0.16485384]],
       dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(10,) dtype=float32, numpy=array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>)

实施自定义层<br>
实现自己的层的最佳方法是扩展tf.keras.Layer类并实现：* __init__，可以进行所有与输入无关的初始化* build，可以知道输入张量的形状，并可以进行其余的初始化* call，在这里进行正向计算

请注意，您不必等到build被调用来创建变量时，也可以在中创建它们__init__。但是，在其中创建它们的好处build是，它可以根据将在其上进行操作的输入的形状来进行后期变量创建。另一方面，在中创建变量__init__将意味着需要明确指定创建变量所需的形状。

In [7]:
class MyDenseLayer(tf.keras.layers.Layer):
    def __init__(self, num_outputs):
        super(MyDenseLayer, self).__init__()
        self.num_outputs = num_outputs

    def build(self, input_shape):
        self.kernel = self.add_variable("kernel",
                                    shape=[int(input_shape[-1]),
                                           self.num_outputs])

    def call(self, input):
        return tf.matmul(input, self.kernel)

layer = MyDenseLayer(10)

In [8]:
print([var.name for var in layer.trainable_variables])

[]


In [9]:
_ = layer(tf.zeros([10, 5])) # Calling the layer `.builds` it.

Instructions for updating:
Please use `layer.add_weight` method instead.


In [10]:
print([var.name for var in layer.trainable_variables])

['my_dense_layer/kernel:0']


如果整体代码尽可能使用标准层，则更易于阅读和维护，因为其他读者会熟悉标准层的行为。如果您想使用不存在的图层tf.keras.layers，请考虑提交github问题，或者甚至更好的是，向我们发送请求请求！

型号：组成图层<br>
机器学习模型中许多有趣的类似于层的事物都是通过组合现有层来实现的。例如，resnet中的每个残差块都是卷积，批处理规范化和快捷方式的组合。图层可以嵌套在其他图层中。

通常你继承keras.Model的时候，你需要像模型方法：Model.fit，Model.evaluate，和Model.save（见自定义Keras层和模型的详细信息）。

keras.Model（而不是keras.layers.Layer）提供的另一个功能是，除了跟踪变量之外，a keras.Model还跟踪其内部层，从而使它们更易于检查。

例如，这是一个ResNet块：

In [11]:
class ResnetIdentityBlock(tf.keras.Model):
    def __init__(self, kernel_size, filters):
        super(ResnetIdentityBlock, self).__init__(name='')
        filters1, filters2, filters3 = filters

        self.conv2a = tf.keras.layers.Conv2D(filters1, (1, 1))
        self.bn2a = tf.keras.layers.BatchNormalization()

        self.conv2b = tf.keras.layers.Conv2D(filters2, kernel_size, padding='same')
        self.bn2b = tf.keras.layers.BatchNormalization()

        self.conv2c = tf.keras.layers.Conv2D(filters3, (1, 1))
        self.bn2c = tf.keras.layers.BatchNormalization()

    def call(self, input_tensor, training=False):
        x = self.conv2a(input_tensor)
        x = self.bn2a(x, training=training)
        x = tf.nn.relu(x)

        x = self.conv2b(x)
        x = self.bn2b(x, training=training)
        x = tf.nn.relu(x)

        x = self.conv2c(x)
        x = self.bn2c(x, training=training)

        x += input_tensor
        return tf.nn.relu(x)


block = ResnetIdentityBlock(1, [1, 2, 3])

In [13]:
block(tf.ones([1, 2, 3, 3])) 

<tf.Tensor: id=354, shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]],

        [[1., 1., 1.],
         [1., 1., 1.],
         [1., 1., 1.]]]], dtype=float32)>

In [12]:
block.layers

In [13]:
len(block.variables)

18

In [14]:
block.summary()

Model: "resnet_identity_block"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  4         
_________________________________________________________________
batch_normalization (BatchNo multiple                  4         
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  4         
_________________________________________________________________
batch_normalization_1 (Batch multiple                  8         
_________________________________________________________________
conv2d_2 (Conv2D)            multiple                  9         
_________________________________________________________________
batch_normalization_2 (Batch multiple                  12        
Total params: 41
Trainable params: 29
Non-trainable params: 12
________________________________________________

In [19]:
def ResnetIdentityBlockS(x):
    return x+tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1),
                                                    input_shape=(
                                                        None, None, 3)),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(2, 1,
                                                    padding='same'),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(3, (1, 1)),
                             tf.keras.layers.BatchNormalization()])(x)
    

ResnetIdentityBlockS(tf.zeros([1, 2, 3, 3]))

<tf.Tensor: id=1304, shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>

但是，在很多时候，组成多层的模型只是简单地将一层称为另一层。这可以使用tf.keras.Sequential以下代码完成：

In [20]:
my_seq = tf.keras.Sequential([tf.keras.layers.Conv2D(1, (1, 1),
                                                    input_shape=(
                                                        None, None, 3)),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(2, 1,
                                                    padding='same'),
                             tf.keras.layers.BatchNormalization(),
                             tf.keras.layers.Conv2D(3, (1, 1)),
                             tf.keras.layers.BatchNormalization()])
my_seq(tf.zeros([1, 2, 3, 3]))

<tf.Tensor: id=1752, shape=(1, 2, 3, 3), dtype=float32, numpy=
array([[[[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]],

        [[0., 0., 0.],
         [0., 0., 0.],
         [0., 0., 0.]]]], dtype=float32)>

In [21]:
my_seq.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, None, None, 1)     4         
_________________________________________________________________
batch_normalization_9 (Batch (None, None, None, 1)     4         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, None, None, 2)     4         
_________________________________________________________________
batch_normalization_10 (Batc (None, None, None, 2)     8         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, None, None, 3)     9         
_________________________________________________________________
batch_normalization_11 (Batc (None, None, None, 3)     12        
Total params: 41
Trainable params: 29
Non-trainable params: 12
_________________________________________________________

下一步<br>
现在，您可以回到上一个笔记本，并修改线性回归示例以使用图层和模型来更好地进行结构化。